In [ ]:
import numpy as np
import cv2
import os
import sys

# Environment variables or hardcoded for testing
img_path1 = os.getenv('IMAGE1_PATH')
img_path2 = os.getenv('IMAGE2_PATH')
output_path = os.getenv('OUTPUT_PATH', 'output_video.mp4')

# Ensure the output directory exists
output_dir = 'output_frames'
os.makedirs(output_dir, exist_ok=True)

# Read images
img1 = cv2.imread(img_path1)
img2 = cv2.imread(img_path2)

# Ensure both images are the same size
if img1.shape[:2] != img2.shape[:2]:
    img2 = cv2.resize(img2, (img1.shape[1], img1.shape[0]))

# Define points (assumed pre-determined or calculated elsewhere)
pts1 = np.array([[218, 240], [295, 240], [250, 383]], np.float32)
pts2 = np.array([[248, 245], [345, 270], [281, 366]], np.float32)

# Number of frames for morphing
dis = 100.0

# Pre-calculate transformations
for i in range(int(dis)):
    pts11 = pts1 + (pts2 - pts1) * (i / dis)
    pts22 = pts2 - (pts2 - pts1) * (i / dis)
    
    # Affine transformations
    mat1 = cv2.getAffineTransform(pts1, pts11)
    mat2 = cv2.getAffineTransform(pts2, pts22)
    
    # Apply transformations
    dst1 = cv2.warpAffine(img1, mat1, (img1.shape[1], img1.shape[0]), borderMode=cv2.BORDER_REPLICATE)
    dst2 = cv2.warpAffine(img2, mat2, (img2.shape[1], img2.shape[0]), borderMode=cv2.BORDER_REPLICATE)
    
    # Blend images
    alpha = i / dis
    dst = cv2.addWeighted(dst1, 1 - alpha, dst2, alpha, 0)
    
    # Save the resulting frame
    cv2.imwrite(os.path.join(output_dir, f'frame_{i:04d}.png'), dst)

# Create a video from images
fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
video = cv2.VideoWriter(output_path, fourcc, 10, (img1.shape[1], img1.shape[0]))

# Load all frames and write to video
for i in range(int(dis)):
    frame = cv2.imread(os.path.join(output_dir, f'frame_{i:04d}.png'))
    video.write(frame)

video.release()
